# Goal

In this tutorial we'll generate a dataset and focus on reproducability.These are the steps:

1. Generate dataset of tfrecords
2. Store information on HOW they were generated
3. Push them to remote (S3 or definiens network share)
4. Access dataset via unique URL

## Generate dataset of tfrecords

In [5]:
from pathlib import Path
DATA_DIRPATH = Path().resolve() / "data"
DATA_DIRPATH.mkdir(exist_ok=True)
DATA_DIRPATH

PosixPath('/home/ubuntu/git/litte_wandb/data')

In [8]:
# see https://gitlab.img.astrazeneca.net/RA/digipath-sdk/-/tree/master/src/simple-tfrecords
import numpy as np
from az_cp_simple_tfrecords import encode,decode, features, infer_description_from_dataset, infer_description_from_record
import tensorflow as tf

def _create_dummy_record():
    x = np.random.randint(0,256,size=(1,2,3), dtype=np.uint8)
    y = np.random.randint(0,10_000,size=(1,2,2,3), dtype=np.uint32)

    dct = {
        "od_value": features.Float32Feature(1.23),
        "bounding_boxes": features.Float32ListFeature([0., 0., 100.,200.]),
        "cell_count": features.Int64Feature(2**60),
        "slide_name": features.StringFeature("critical slide"),
        "low_resolution_image": features.TFPNGFeature(x),
        "multichannel_image": features.TFArrayFeature(tf.convert_to_tensor(y)),
    }
    example = encode(dct)
    return example.SerializeToString()


# We'll create 3 tfrecords file with 5 records each.
for i in range(3):
    tfrecord_path= DATA_DIRPATH / f"train_{i}.tfrecord"
    with tf.io.TFRecordWriter(str(tfrecord_path)) as writer:
        for _ in range(5):
            record = _create_dummy_record()
            writer.write(record)


## Store information on HOW data was generated

In [11]:
from little_wandb import dump_infos

dump_infos(DATA_DIRPATH / "repro")

wandb: WARNING Invalid value for property files_dir: /home/ubuntu/git/litte_wandb/data/repro. This will raise an error in the future.
unable to save code -- program entry not found


In [17]:
for p in reversed(sorted(DATA_DIRPATH.glob("repro/*"))):
    print(f"\n\n### Content of {p}")
    print(p.read_text())




### Content of /home/ubuntu/git/litte_wandb/data/repro/wandb-metadata.json
{
    "os": "Linux-5.19.0-1024-aws-x86_64-with-glibc2.35",
    "python": "3.10.6",
    "heartbeatAt": "2023-05-11T14:34:37.579951",
    "startedAt": "2023-05-11T14:34:37.577779",
    "docker": null,
    "cuda": null,
    "args": null,
    "state": "running",
    "program": "<python with no main file>",
    "git": {
        "remote": "git@gitlab.img.astrazeneca.net:RA/experimental/litte_wandb.git",
        "commit": "bb9d942d7480a21380594da0be0a9bfff624f4b1"
    },
    "email": "",
    "root": "/home/ubuntu/git/litte_wandb",
    "host": "cgebbe-r6id.img.astrazeneca.net",
    "username": "ubuntu",
    "executable": "/home/ubuntu/git/litte_wandb/venv/bin/python"
}



### Content of /home/ubuntu/git/litte_wandb/data/repro/requirements.txt
ansiwrap==0.8.4
appdirs==1.4.4
asttokens==2.2.1
attrs==23.1.0
backcall==0.2.0
build==0.10.0
certifi==2023.5.7
charset-normalizer==3.1.0
click==8.1.3
comm==0.1.3
debugpy==1.6.7
de